# Capstone Project - The Battle of Neighborhoods!

Install and import required packages

In [15]:
# install the Google Trends API
# !pip install pytrends

# install the Daft Listings API
!pip install daftlistings

# install the Daft Scraper API
!pip install daft-scraper==1.2.7

# install geopandas, geopy
!pip install geopandas
!pip install geopy

# install folium
!pip install folium

# install matplotlib
!pip install matplotlib

# install pandas profiling
!pip install pandas-profiling==2.7.1

     |████████████████████████████████| 59 kB 1.4 MB/s eta 0:00:011
  Attempting uninstall: daft-scraper
    Found existing installation: daft-scraper 1.3.0
    Uninstalling daft-scraper-1.3.0:
      Successfully uninstalled daft-scraper-1.3.0


In [2]:
# python packages
import pprint
import requests
import geopandas
import pyproj as pp
import numpy as np
import pandas as pd
import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Google Trends API packages
from pytrends.request import TrendReq

# Daft listings API packages
from daftlistings import Daft, RentType, SortOrder, SortType, MapVisualization, SaleType
from joblib import Parallel, delayed
import time

# Daft Scraper API packages
from daft_scraper.search import DaftSearch, SearchType
from daft_scraper.search.options import (PropertyType, PropertyTypesOption, AdState, AdStateOption, PriceOption, BedOption)
from daft_scraper.search.options_location import Location, LocationsOption

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Pandas Profiling
import pandas_profiling as pd_prof

# sklearn import for DBSCAN clustering
from sklearn.cluster import DBSCAN

## 1. Introduction
This section outlines a general background for the Business Problem that I'll be trying to solve as part of the capstone project.

The primary focus for this project would be on the city Dublin and its 22 different District areas.  

This project tries to achieve the following analyses for the respective target audience in mind:  
1) **House Renting**: Finding your ideal house/apartment to buy/invest in Dublin city is very challenging given the housing crisis in Ireland and more specifically in the capital city of Dublin. The target audience in this case is people looking for properties for sale in the city. The attempt here is to filter out properties based on user preferences for apartment characteristics, neighborhood choices in terms of amenities and pricing in the neighborhood in which the property is situated.  
2) **Neighborhood Clustering**: The approach here is to use visualization techniques to cluster districts within Dublin city using clustering techniques based on the venues and venue categories present in different districts. We can get a sense of how different districts are oriented within the city in terms of different places, amenities, transport routes and most importantly whether distance from the city centre plays a role in driving this.    

Overall the aim is to aid people looking for buying houses in Dublin city and help them filter out neighborhoods and properties based on their preferences as well as other local factors driving their decision making.  
Apart from that, the visualiztion techniques used for analysing different datasets would help certain stakeholders make decisions in terms of government planning, business marketing decisions as well as general readers looking for some insights of their own city! 

## 2. Data
This section defines the different data sources as well as their sample examples that have been used for this assignment.

### 1) Daft Listings API
As seen below, this is a very useful API [https://github.com/TheJokersThief/daft-scraper] which is simple to use and get up to speed.  
The sample example above shows a search using the API filter options to get all listings in "Dublin county where a property sale has been agreed for any type of property" as listed on the popular Irish Property website (Daft.ie)  
We fetch all such listings and build a dataframe containing all the useful features for each property which as seen below would consist of <price', 'facilities', 'address', 'num_bedrooms', 'num_bathrooms', 'latitude', 'longitude'>  
This data would help us recommend properties to the targeted end-user as well as the geographical coordinates would help us visually analyse the data in question.  

### 2) Foursquare Places API
Finally, the last part involves a similar approach taken during the previous weeks in this course where we had analysed different neighborhoods in Toronto, Canada.  
The challenge here is to obtain different districts comprising within Dublin City and obtain their respectice geographical coordinates using Nominatim geolocator.  
The sample code given below shows how we plan to construct the final dataframe where each row would be an individual venue along-with the attributes of each of the venues including their geolcation coordinates.  
OneHotEncoding can be used to get a feature representing distribution of different types of venues as well as the most popular and dominating venue type in each of the districts within Dublin city.  

## 3. Code
This section below has all the step-by-step code statements with comments added for explicit explanation wherever necessary.

In [2]:
# exploring the daft scraper API for the latest version of Daft.ie

In [3]:
# call to the API for fetching all SALE AGREED properties in Dublin
options = [
    PropertyTypesOption([PropertyType.ALL]),
    LocationsOption([Location.DUBLIN_COUNTY, Location.DUBLIN_CITY, Location.DUBLIN_CITY_CENTRE_DUBLIN, Location.DUBLIN_COMMUTER_TOWNS_DUBLIN, Location.DUN_LAOGHAIRE_DUBLIN]),
    AdStateOption(AdState.AVAILABLE),
    BedOption(5,5)
]

api = DaftSearch(SearchType.RENT)
listings = api.search(options)

In [4]:
print(len(listings))

cnt_price = 0
cnt_abr_price = 0

for listing in listings:
    if hasattr(listing, 'price'):
        cnt_price += 1
    if hasattr(listing, 'abbreviatedPrice'):
        cnt_abr_price += 1

print(cnt_price, cnt_abr_price)

40
40 40


In [5]:
test_df2 = pd.DataFrame([vars(f) for f in listings])

In [6]:
test_df2.head()

,price,category,_id,state,seller,abbreviatedPrice,numBedrooms,daftShortcode,prs,ber,point,saleType,seoTitle,propertyType,sections,title,seoFriendlyPath,publishDate,media,featuredLevel,pageBranding,url,numBathrooms,sticker
0,711.00,Rent,2606278,PUBLISHED,"{'sellerId': 12035, 'sellerType': 'BRANDED_AGE...",€710+,1,9196526,"{'tagLine': 'Dublin's Most Central Address', '...",{'rating': 'NA'},"{'coordinates': [-6.25013779547416, 53.3537148...",[To Let],"Buckley Hall, Buckingham Street Lower, Dublin 1",Apartments,"[Property, Private Rental Sector, Apartments]","Buckley Hall, Buckingham Street Lower, Dublin 1",/for-rent/buckley-hall-buckingham-street-lower...,1617372789000,"{'hasBrochure': False, 'images': [{'size720x48...",PREMIUM,"{'backgroundColour': '#eeeeee', 'squareLogos':...",https://www.daft.ie//for-rent/buckley-hall-buc...,NaN,NaN
1,429.66,Rent,2580818,PUBLISHED,"{'sellerId': 8497, 'sellerType': 'BRANDED_AGEN...",€100+,1,9193608,"{'tagLine': 'Save Money - Same Experience', 'l...",{'rating': 'NA'},"{'coordinates': [-6.305145019049149, 53.365027...",[To Let],"JBs Student Campus, Dublin 7, JB's Student Campus",Apartments,"[Property, Private Rental Sector, Apartments]","JBs Student Campus, Dublin 7, JB's Student Campus",/for-rent/jbs-student-campus-dublin-7-jbs-stud...,1617287666000,"{'hasBrochure': False, 'images': [{'size720x48...",PREMIUM,"{'backgroundColour': '#eeeeee', 'squareLogos':...",https://www.daft.ie//for-rent/jbs-student-camp...,NaN,NaN
2,5000.00,Rent,3154857,PUBLISHED,"{'sellerId': 5130226, 'sellerType': 'PRIVATE_U...","€5,000",5,24169856,NaN,{'rating': 'SI_666'},"{'coordinates': [-6.5867589, 53.375964], 'poin...",[To Let],"124 Railpark, Maynooth, Co. Kildare",House,"[Property, Residential, House]","124 Railpark, Maynooth, Co. Kildare",/for-rent/house-124-railpark-maynooth-co-kilda...,1617181599141,"{'hasBrochure': False, 'images': [{'caption': ...",FEATURED,NaN,https://www.daft.ie//for-rent/house-124-railpa...,6.0,NaN
3,3500.00,Rent,2942769,PUBLISHED,{'address': 'Unit 12 Knockmitten Business Par...,"€3,500",5,23974899,NaN,{'rating': 'A3'},"{'coordinates': [-6.136831, 53.494352], 'point...",[To Let],"2 Rahillion Drive, Donabate, Co. Dublin",House,"[Property, Residential, House]","Rahillion Drive, Donabate, Co. Dublin",/for-rent/house-rahillion-drive-donabate-co-du...,1617349845094,"{'hasBrochure': False, 'images': [{'size720x48...",PREMIUM,{'standardLogo': 'https://photos.cdn.dsch.ie/N...,https://www.daft.ie//for-rent/house-rahillion-...,4.0,NaN
4,2700.00,Rent,2588136,PUBLISHED,"{'address': 'Lucan Co. Dublin', 'sellerId': 8...","€2,700",5,22075224,NaN,{'rating': 'C1'},"{'coordinates': [-6.4532423, 53.3632534], 'poi...",[To Let],"42 Laragchon, Lucan, Co. Dublin",House,"[Property, Residential, House]","42 Laragchon, Lucan, Co. Dublin",/for-rent/house-42-laragchon-lucan-co-dublin/2...,1617270787188,"{'hasBrochure': False, 'images': [{'size720x48...",PREMIUM,{'standardLogo': 'https://photos.cdn.dsch.ie/Z...,https://www.daft.ie//for-rent/house-42-laragch...,3.0,NaN


In [19]:
# only keep columns of interest
final_df = test_df2[['title', 'propertyType', 'category', 'numBedrooms', 'numBathrooms', 'price', 'abbreviatedPrice', 'ber', 'point', 'publishDate', 'seller']]

In [20]:
final_df.head()

,title,propertyType,category,numBedrooms,numBathrooms,price,abbreviatedPrice,ber,point,publishDate,seller
0,"Buckley Hall, Buckingham Street Lower, Dublin 1",Apartments,Rent,1,NaN,711.00,€710+,{'rating': 'NA'},"{'coordinates': [-6.25013779547416, 53.3537148...",1617372789000,"{'sellerId': 12035, 'sellerType': 'BRANDED_AGE..."
1,"JBs Student Campus, Dublin 7, JB's Student Campus",Apartments,Rent,1,NaN,429.66,€100+,{'rating': 'NA'},"{'coordinates': [-6.305145019049149, 53.365027...",1617287666000,"{'sellerId': 8497, 'sellerType': 'BRANDED_AGEN..."
2,"124 Railpark, Maynooth, Co. Kildare",House,Rent,5,6.0,5000.00,"€5,000",{'rating': 'SI_666'},"{'coordinates': [-6.5867589, 53.375964], 'poin...",1617181599141,"{'sellerId': 5130226, 'sellerType': 'PRIVATE_U..."
3,"Rahillion Drive, Donabate, Co. Dublin",House,Rent,5,4.0,3500.00,"€3,500",{'rating': 'A3'},"{'coordinates': [-6.136831, 53.494352], 'point...",1617349845094,{'address': 'Unit 12 Knockmitten Business Par...
4,"42 Laragchon, Lucan, Co. Dublin",House,Rent,5,3.0,2700.00,"€2,700",{'rating': 'C1'},"{'coordinates': [-6.4532423, 53.3632534], 'poi...",1617270787188,"{'address': 'Lucan Co. Dublin', 'sellerId': 8..."


In [21]:
# logic to fetch the neighbourhood for each row depending on the number of tokens as part of the title split
new = final_df["title"].str.split(",", n = 4, expand = True) 

new[3].fillna(new[2], inplace=True)
new[3].fillna(new[1], inplace=True)
new[3].fillna(new[0], inplace=True)
# new[3].fillna(new[0], inplace=True)
# new[5].fillna(new[0], inplace=True)

In [22]:
final_df["neighbourhood"]= new[3] 

/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
# filter out neighbourhoods with low cardinality
col = 'neighbourhood'
n = 10
final_df = final_df[final_df.groupby(col)[col].transform('count').ge(n)]

vague_n = final_df[final_df['neighbourhood'] == ' Co. Dublin'].index
final_df.drop(vague_n , inplace=True)

In [23]:
final_df.groupby(['neighbourhood']).size()

neighbourhood
 Co. Dublin             10
 Co. Kildare             4
 Co. Louth               2
 Co. Meath               2
 Dublin 1                2
 Dublin 15               4
 Dublin 18               4
 Dublin 22               2
 Dublin 4                4
 Dublin 9                4
 JB's Student Campus     2
dtype: int64

In [13]:
len(final_df)

2136

In [14]:
# logic to replace 0 price values with corresponding figures from abbreviatedPrice column
final_df['val'] = final_df['abbreviatedPrice'].str.replace('€','')
final_df['val'] = final_df['val'].str.replace('+','')
final_df['val'] = final_df['val'].str.replace('POA','0')

final_df.val = (final_df.val.replace(r'[kM]+$', '', regex=True).astype(float) * final_df.val.str.extract(r'[\d\.]+([kM]+)', expand=False).fillna(1).replace(['k','M'], [10**3, 10**6]).astype(int))

final_df.price.fillna(final_df.val, inplace=True)

/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
final_df.head()

,title,propertyType,category,numBedrooms,numBathrooms,price,abbreviatedPrice,ber,point,publishDate,seller,floorArea,neighbourhood,val
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Apartment,Buy,2.0,2.0,200000.0,€200k,"{'code': '113646038', 'rating': 'C1', 'epi': '...","{'coordinates': [-6.392909, 53.422384], 'point...",1616077963000,"{'address': 'Unit 4, The Plaza, Tyrrelstown ...","{'unit': 'METRES_SQUARED', 'value': '65'}",Dublin 15,200000.0
1,"84 Rossfield Park, Tallaght, Dublin 24",End of Terrace,Buy,4.0,1.0,190000.0,€190k,NaN,"{'coordinates': [-6.400766, 53.288486], 'point...",1616809150000,"{'address': 'Dublin Office, Land & Property, ...",NaN,Dublin 24,190000.0
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",End of Terrace,Buy,3.0,1.0,260000.0,€260k,"{'code': '110834496', 'rating': 'E2', 'epi': '...","{'coordinates': [-6.319989, 53.384121], 'point...",1615453791000,"{'address': 'Unit 4, The Plaza, Tyrrelstown ...","{'unit': 'METRES_SQUARED', 'value': '95'}",Dublin 11,260000.0
3,"Highfield, Mornington Park, Malahide Road, Art...",Bungalow,Buy,3.0,3.0,495000.0,€495k,"{'code': '109681171', 'rating': 'C3', 'epi': '...","{'coordinates': [-6.208588, 53.381566], 'point...",1616861068000,{'address': '183 Howth Road Killester Dublin...,"{'unit': 'METRES_SQUARED', 'value': '118'}",Dublin 5,495000.0
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Apartment,Buy,2.0,2.0,340000.0,€340k,{'rating': 'B3'},"{'coordinates': [-6.337658, 53.373575], 'point...",1616057065000,{'address': 'Unit 3 Phoenix Park Way Phoenix ...,"{'unit': 'METRES_SQUARED', 'value': '75'}",Dublin 15,340000.0


In [16]:
# remove rows with 0 price
zero_val = final_df[final_df['val'] == 0].index
final_df.drop(zero_val , inplace=True)

final_df.groupby(['val']).size()

val
75000.0       1
129000.0      1
135000.0      1
139000.0      2
140000.0      4
145000.0      1
150000.0      6
159000.0      1
160000.0      5
165000.0      5
169000.0      1
170000.0      9
175000.0      8
179000.0      4
180000.0      8
185000.0      7
188000.0      1
189000.0      2
190000.0     14
194000.0      1
195000.0     20
197000.0      1
199000.0     11
200000.0     28
205000.0      2
209000.0      4
210000.0     16
212000.0      1
215000.0     24
219000.0      1
220000.0     28
223000.0      1
225000.0     57
229000.0      4
230000.0     25
235000.0     34
238000.0      1
239000.0      2
240000.0     25
245000.0     14
248000.0      1
249000.0      9
250000.0     59
255000.0     11
259000.0      6
260000.0     30
265000.0     30
267000.0      1
269000.0      6
270000.0     25
275000.0     68
279000.0      6
280000.0     17
285000.0     35
286000.0      1
288000.0      1
289000.0      2
290000.0     21
295000.0     62
298000.0      1
299000.0      9
300000.0     45
3050

In [17]:
test_df3 = pd.concat([final_df.drop(['seller'], axis=1), final_df['seller'].apply(pd.Series)], axis=1)

test_df3 = pd.concat([test_df3.drop(['floorArea'], axis=1), test_df3['floorArea'].apply(pd.Series)], axis=1)

In [18]:
test_df3.head()

,title,propertyType,category,numBedrooms,numBathrooms,price,abbreviatedPrice,ber,point,publishDate,neighbourhood,val,address,standardLogo,showContactForm,branch,phoneWhenToCall,squareLogo,backgroundColour,sellerId,phone,name,sellerType,licenceNumber,alternativePhone,profileImage,0,unit,value
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Apartment,Buy,2.0,2.0,200000.0,€200k,"{'code': '113646038', 'rating': 'C1', 'epi': '...","{'coordinates': [-6.392909, 53.422384], 'point...",1616077963000,Dublin 15,200000.0,"Unit 4,\r\nThe Plaza,\r\nTyrrelstown Town Cent...",https://photos.cdn.dsch.ie/Njc5MDk5N2NlYWFiN2J...,True,McPeake Auctioneers,9.00 - 5.30 Mon. - Fri.,https://photos.cdn.dsch.ie/OWIyOTg5NjFlZjBkZGV...,#f77b00,1815,01 827 2300,Hilary Cummins,BRANDED_AGENT,001012,NaN,NaN,NaN,METRES_SQUARED,65
1,"84 Rossfield Park, Tallaght, Dublin 24",End of Terrace,Buy,4.0,1.0,190000.0,€190k,NaN,"{'coordinates': [-6.400766, 53.288486], 'point...",1616809150000,Dublin 24,190000.0,"Dublin Office,\r\nLand & Property,\r\n27 Lower...",https://photos.cdn.dsch.ie/ODlmOGNiNmFhZjhkMzI...,True,BRG Gibson Auctions,9am - 5:30pm,https://photos.cdn.dsch.ie/ZDAyZjMwM2U1ZWM0M2F...,#000000,9086,01 447 5177,BRG Gibson,BRANDED_AGENT,004031,NaN,NaN,NaN,NaN,NaN
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",End of Terrace,Buy,3.0,1.0,260000.0,€260k,"{'code': '110834496', 'rating': 'E2', 'epi': '...","{'coordinates': [-6.319989, 53.384121], 'point...",1615453791000,Dublin 11,260000.0,"Unit 4,\r\nThe Plaza,\r\nTyrrelstown Town Cent...",https://photos.cdn.dsch.ie/Njc5MDk5N2NlYWFiN2J...,True,McPeake Auctioneers,9.00 - 5.30 Mon. - Fri.,https://photos.cdn.dsch.ie/OWIyOTg5NjFlZjBkZGV...,#f77b00,1815,01 827 2300,Hilary Cummins,BRANDED_AGENT,001012,NaN,NaN,NaN,METRES_SQUARED,95
3,"Highfield, Mornington Park, Malahide Road, Art...",Bungalow,Buy,3.0,3.0,495000.0,€495k,"{'code': '109681171', 'rating': 'C3', 'epi': '...","{'coordinates': [-6.208588, 53.381566], 'point...",1616861068000,Dublin 5,495000.0,183 Howth Road\r\nKillester\r\nDublin 3,https://photos.cdn.dsch.ie/YjEyMzk4NzQ4MTM2ZmN...,True,Sherry FitzGerald Killester,NaN,https://photos.cdn.dsch.ie/MTIyMGQyYjA3NmE5MTZ...,#0f3a5d,2646,01 833 6555,Jason Kavanagh,BRANDED_AGENT,002183,087 9255198,NaN,NaN,METRES_SQUARED,118
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Apartment,Buy,2.0,2.0,340000.0,€340k,{'rating': 'B3'},"{'coordinates': [-6.337658, 53.373575], 'point...",1616057065000,Dublin 15,340000.0,Unit 3 Phoenix Park Way\r\nPhoenix Park Raceco...,https://photos.cdn.dsch.ie/MzFjMWE2NTI5ODZmYzg...,True,SATIS PROPERTY,"Monday - Friday 8:30 am - 5:00pm, Closed for L...",https://photos.cdn.dsch.ie/YWMwZjU1MmM4MGJkMGV...,#192b6b,4274,01 829 9150,Karen Carberry,BRANDED_AGENT,002943,NaN,https://photos.cdn.dsch.ie/MzcwYTRmMTBiM2RhYzg...,NaN,METRES_SQUARED,75


In [19]:
test_df3['value'] = test_df3['value'].where(test_df3['unit'] == 'METRES_SQUARED', test_df3['value'].astype(float) * 4046.86)

In [20]:
test_df3.groupby(['value']).size()

value
121.4058               1
202.34300000000002     3
404.68600000000004     1
445.1546               1
607.029                1
809.3720000000001      1
1011.715               1
1092.6522              1
1214.058               1
3480.2996000000003     1
3601.7054000000003     1
4046.86                1
6596.3818              1
100                   27
101                   16
102                   21
103                   22
104                    9
105                   25
106                   12
107                   11
108                    7
109                   13
110                   18
111                    9
112                   12
113                    6
114                   10
115                   10
116                   16
117                    6
118                   11
119                    6
120                    9
121                    8
122                    7
123                   10
124                   10
125                   15
126                

In [21]:
# split out the point column into (long, lat) values as 2 new columns
test_df3 = pd.concat([test_df3.drop(['point'], axis=1), test_df3['point'].apply(pd.Series)], axis=1)

test_df3[['longitude','latitude']] = pd.DataFrame(test_df3.coordinates.tolist(), index=test_df3.index)

In [22]:
test_df3.head()

,title,propertyType,category,numBedrooms,numBathrooms,price,abbreviatedPrice,ber,publishDate,neighbourhood,val,address,standardLogo,showContactForm,branch,phoneWhenToCall,squareLogo,backgroundColour,sellerId,phone,name,sellerType,licenceNumber,alternativePhone,profileImage,0,unit,value,coordinates,point_type,longitude,latitude
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Apartment,Buy,2.0,2.0,200000.0,€200k,"{'code': '113646038', 'rating': 'C1', 'epi': '...",1616077963000,Dublin 15,200000.0,"Unit 4,\r\nThe Plaza,\r\nTyrrelstown Town Cent...",https://photos.cdn.dsch.ie/Njc5MDk5N2NlYWFiN2J...,True,McPeake Auctioneers,9.00 - 5.30 Mon. - Fri.,https://photos.cdn.dsch.ie/OWIyOTg5NjFlZjBkZGV...,#f77b00,1815,01 827 2300,Hilary Cummins,BRANDED_AGENT,001012,NaN,NaN,NaN,METRES_SQUARED,65,"[-6.392909, 53.422384]",Point,-6.392909,53.422384
1,"84 Rossfield Park, Tallaght, Dublin 24",End of Terrace,Buy,4.0,1.0,190000.0,€190k,NaN,1616809150000,Dublin 24,190000.0,"Dublin Office,\r\nLand & Property,\r\n27 Lower...",https://photos.cdn.dsch.ie/ODlmOGNiNmFhZjhkMzI...,True,BRG Gibson Auctions,9am - 5:30pm,https://photos.cdn.dsch.ie/ZDAyZjMwM2U1ZWM0M2F...,#000000,9086,01 447 5177,BRG Gibson,BRANDED_AGENT,004031,NaN,NaN,NaN,NaN,NaN,"[-6.400766, 53.288486]",Point,-6.400766,53.288486
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",End of Terrace,Buy,3.0,1.0,260000.0,€260k,"{'code': '110834496', 'rating': 'E2', 'epi': '...",1615453791000,Dublin 11,260000.0,"Unit 4,\r\nThe Plaza,\r\nTyrrelstown Town Cent...",https://photos.cdn.dsch.ie/Njc5MDk5N2NlYWFiN2J...,True,McPeake Auctioneers,9.00 - 5.30 Mon. - Fri.,https://photos.cdn.dsch.ie/OWIyOTg5NjFlZjBkZGV...,#f77b00,1815,01 827 2300,Hilary Cummins,BRANDED_AGENT,001012,NaN,NaN,NaN,METRES_SQUARED,95,"[-6.319989, 53.384121]",Point,-6.319989,53.384121
3,"Highfield, Mornington Park, Malahide Road, Art...",Bungalow,Buy,3.0,3.0,495000.0,€495k,"{'code': '109681171', 'rating': 'C3', 'epi': '...",1616861068000,Dublin 5,495000.0,183 Howth Road\r\nKillester\r\nDublin 3,https://photos.cdn.dsch.ie/YjEyMzk4NzQ4MTM2ZmN...,True,Sherry FitzGerald Killester,NaN,https://photos.cdn.dsch.ie/MTIyMGQyYjA3NmE5MTZ...,#0f3a5d,2646,01 833 6555,Jason Kavanagh,BRANDED_AGENT,002183,087 9255198,NaN,NaN,METRES_SQUARED,118,"[-6.208588, 53.381566]",Point,-6.208588,53.381566
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Apartment,Buy,2.0,2.0,340000.0,€340k,{'rating': 'B3'},1616057065000,Dublin 15,340000.0,Unit 3 Phoenix Park Way\r\nPhoenix Park Raceco...,https://photos.cdn.dsch.ie/MzFjMWE2NTI5ODZmYzg...,True,SATIS PROPERTY,"Monday - Friday 8:30 am - 5:00pm, Closed for L...",https://photos.cdn.dsch.ie/YWMwZjU1MmM4MGJkMGV...,#192b6b,4274,01 829 9150,Karen Carberry,BRANDED_AGENT,002943,NaN,https://photos.cdn.dsch.ie/MzcwYTRmMTBiM2RhYzg...,NaN,METRES_SQUARED,75,"[-6.337658, 53.373575]",Point,-6.337658,53.373575


In [23]:
# similarly split the ber column to fetch the rating
test_df3 = pd.concat([test_df3.drop(['ber'], axis=1), test_df3['ber'].apply(pd.Series)], axis=1)

In [24]:
test_df3.head()

,title,propertyType,category,numBedrooms,numBathrooms,price,abbreviatedPrice,publishDate,neighbourhood,val,address,standardLogo,showContactForm,branch,phoneWhenToCall,squareLogo,backgroundColour,sellerId,phone,name,sellerType,licenceNumber,alternativePhone,profileImage,0,unit,value,coordinates,point_type,longitude,latitude,0,code,epi,rating
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Apartment,Buy,2.0,2.0,200000.0,€200k,1616077963000,Dublin 15,200000.0,"Unit 4,\r\nThe Plaza,\r\nTyrrelstown Town Cent...",https://photos.cdn.dsch.ie/Njc5MDk5N2NlYWFiN2J...,True,McPeake Auctioneers,9.00 - 5.30 Mon. - Fri.,https://photos.cdn.dsch.ie/OWIyOTg5NjFlZjBkZGV...,#f77b00,1815,01 827 2300,Hilary Cummins,BRANDED_AGENT,001012,NaN,NaN,NaN,METRES_SQUARED,65,"[-6.392909, 53.422384]",Point,-6.392909,53.422384,NaN,113646038,167.18 kWh/m2/yr,C1
1,"84 Rossfield Park, Tallaght, Dublin 24",End of Terrace,Buy,4.0,1.0,190000.0,€190k,1616809150000,Dublin 24,190000.0,"Dublin Office,\r\nLand & Property,\r\n27 Lower...",https://photos.cdn.dsch.ie/ODlmOGNiNmFhZjhkMzI...,True,BRG Gibson Auctions,9am - 5:30pm,https://photos.cdn.dsch.ie/ZDAyZjMwM2U1ZWM0M2F...,#000000,9086,01 447 5177,BRG Gibson,BRANDED_AGENT,004031,NaN,NaN,NaN,NaN,NaN,"[-6.400766, 53.288486]",Point,-6.400766,53.288486,NaN,NaN,NaN,NaN
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",End of Terrace,Buy,3.0,1.0,260000.0,€260k,1615453791000,Dublin 11,260000.0,"Unit 4,\r\nThe Plaza,\r\nTyrrelstown Town Cent...",https://photos.cdn.dsch.ie/Njc5MDk5N2NlYWFiN2J...,True,McPeake Auctioneers,9.00 - 5.30 Mon. - Fri.,https://photos.cdn.dsch.ie/OWIyOTg5NjFlZjBkZGV...,#f77b00,1815,01 827 2300,Hilary Cummins,BRANDED_AGENT,001012,NaN,NaN,NaN,METRES_SQUARED,95,"[-6.319989, 53.384121]",Point,-6.319989,53.384121,NaN,110834496,364.09 kWh/m2/yr,E2
3,"Highfield, Mornington Park, Malahide Road, Art...",Bungalow,Buy,3.0,3.0,495000.0,€495k,1616861068000,Dublin 5,495000.0,183 Howth Road\r\nKillester\r\nDublin 3,https://photos.cdn.dsch.ie/YjEyMzk4NzQ4MTM2ZmN...,True,Sherry FitzGerald Killester,NaN,https://photos.cdn.dsch.ie/MTIyMGQyYjA3NmE5MTZ...,#0f3a5d,2646,01 833 6555,Jason Kavanagh,BRANDED_AGENT,002183,087 9255198,NaN,NaN,METRES_SQUARED,118,"[-6.208588, 53.381566]",Point,-6.208588,53.381566,NaN,109681171,206.41 kWh/m2/yr,C3
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Apartment,Buy,2.0,2.0,340000.0,€340k,1616057065000,Dublin 15,340000.0,Unit 3 Phoenix Park Way\r\nPhoenix Park Raceco...,https://photos.cdn.dsch.ie/MzFjMWE2NTI5ODZmYzg...,True,SATIS PROPERTY,"Monday - Friday 8:30 am - 5:00pm, Closed for L...",https://photos.cdn.dsch.ie/YWMwZjU1MmM4MGJkMGV...,#192b6b,4274,01 829 9150,Karen Carberry,BRANDED_AGENT,002943,NaN,https://photos.cdn.dsch.ie/MzcwYTRmMTBiM2RhYzg...,NaN,METRES_SQUARED,75,"[-6.337658, 53.373575]",Point,-6.337658,53.373575,NaN,NaN,NaN,B3


In [25]:
# filter out unwanted columns
final_df2 = test_df3[['title', 'neighbourhood', 'propertyType', 'numBedrooms', 'numBathrooms', 'value', 'val', 'rating', 'sellerId', 'longitude', 'latitude', 'publishDate']]

In [26]:
final_df2.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,value,val,rating,sellerId,longitude,latitude,publishDate
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Dublin 15,Apartment,2.0,2.0,65,200000.0,C1,1815,-6.392909,53.422384,1616077963000
1,"84 Rossfield Park, Tallaght, Dublin 24",Dublin 24,End of Terrace,4.0,1.0,NaN,190000.0,NaN,9086,-6.400766,53.288486,1616809150000
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",Dublin 11,End of Terrace,3.0,1.0,95,260000.0,E2,1815,-6.319989,53.384121,1615453791000
3,"Highfield, Mornington Park, Malahide Road, Art...",Dublin 5,Bungalow,3.0,3.0,118,495000.0,C3,2646,-6.208588,53.381566,1616861068000
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Dublin 15,Apartment,2.0,2.0,75,340000.0,B3,4274,-6.337658,53.373575,1616057065000


In [27]:
final_df2[final_df2['numBathrooms'].isnull()].groupby(['propertyType']).size()

propertyType
Apartment     8
Bungalow      2
Detached      5
Houses        2
Semi-D        8
Site         14
Studio        1
Terrace      10
dtype: int64

In [28]:
final_df2.groupby(['rating']).size()

rating
A2         14
A3         30
B1         12
B2         57
B3        139
C1        168
C2        181
C3        203
D1        240
D2        252
E1        185
E2        164
F         136
G         128
SI_666     56
dtype: int64

In [30]:
# handle missing values, replacing with a non-existent constant values
final_df2['numBedrooms'] = final_df2['numBedrooms'].fillna(-1)

final_df2['numBathrooms'] = final_df2['numBathrooms'].fillna(-1)

final_df2['rating'] = final_df2['rating'].fillna('ZZZ')

/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [31]:
final_df2.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,value,val,rating,sellerId,longitude,latitude,publishDate
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Dublin 15,Apartment,2.0,2.0,65,200000.0,C1,1815,-6.392909,53.422384,1616077963000
1,"84 Rossfield Park, Tallaght, Dublin 24",Dublin 24,End of Terrace,4.0,1.0,NaN,190000.0,ZZZ,9086,-6.400766,53.288486,1616809150000
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",Dublin 11,End of Terrace,3.0,1.0,95,260000.0,E2,1815,-6.319989,53.384121,1615453791000
3,"Highfield, Mornington Park, Malahide Road, Art...",Dublin 5,Bungalow,3.0,3.0,118,495000.0,C3,2646,-6.208588,53.381566,1616861068000
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Dublin 15,Apartment,2.0,2.0,75,340000.0,B3,4274,-6.337658,53.373575,1616057065000


In [32]:
mapping_df = final_df2[:100]

mapping_df['n_neigh'] = mapping_df.groupby('neighbourhood').ngroup()

/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
mapping_df.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,value,val,rating,sellerId,longitude,latitude,publishDate,n_neigh
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Dublin 15,Apartment,2.0,2.0,65,200000.0,C1,1815,-6.392909,53.422384,1616077963000,5
1,"84 Rossfield Park, Tallaght, Dublin 24",Dublin 24,End of Terrace,4.0,1.0,NaN,190000.0,ZZZ,9086,-6.400766,53.288486,1616809150000,9
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",Dublin 11,End of Terrace,3.0,1.0,95,260000.0,E2,1815,-6.319989,53.384121,1615453791000,1
3,"Highfield, Mornington Park, Malahide Road, Art...",Dublin 5,Bungalow,3.0,3.0,118,495000.0,C3,2646,-6.208588,53.381566,1616861068000,12
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Dublin 15,Apartment,2.0,2.0,75,340000.0,B3,4274,-6.337658,53.373575,1616057065000,5


In [34]:
# set color scheme for the neighbourhoods
num_neigh = mapping_df.apply(pd.Series.nunique)['neighbourhood']

x = np.arange(num_neigh)
ys = [i + x + (i*x)**2 for i in range(num_neigh)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [35]:
address = 'Dublin, Ireland'

geolocator = Nominatim(user_agent="dublin_locator")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dublin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dublin are 53.3497645, -6.2602732.


In [36]:
# create map of Dublin using latitude and longitude values
map_dublin = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, title, n_neigh in zip(mapping_df['latitude'], mapping_df['longitude'], mapping_df['title'], mapping_df['n_neigh']):
    label = '{}'.format(title)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color=rainbow[n_neigh],
        fill=True,
        fill_color=rainbow[n_neigh],
        fill_opacity=0.7,
        parse_html=False).add_to(map_dublin)  
    
map_dublin

In [37]:
final_df2 = final_df2.rename({'value': 'floorArea', 'val': 'price'}, axis=1)  # new method

final_df2.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,floorArea,price,rating,sellerId,longitude,latitude,publishDate
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Dublin 15,Apartment,2.0,2.0,65,200000.0,C1,1815,-6.392909,53.422384,1616077963000
1,"84 Rossfield Park, Tallaght, Dublin 24",Dublin 24,End of Terrace,4.0,1.0,NaN,190000.0,ZZZ,9086,-6.400766,53.288486,1616809150000
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",Dublin 11,End of Terrace,3.0,1.0,95,260000.0,E2,1815,-6.319989,53.384121,1615453791000
3,"Highfield, Mornington Park, Malahide Road, Art...",Dublin 5,Bungalow,3.0,3.0,118,495000.0,C3,2646,-6.208588,53.381566,1616861068000
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Dublin 15,Apartment,2.0,2.0,75,340000.0,B3,4274,-6.337658,53.373575,1616057065000


In [38]:
# remove rows with NaN for newly added columns
final_df2 = final_df2.dropna()

In [85]:
# add meaningful and useful features
# 1. delta from avg price for that neighbourhood = (avg_price_neighbourhood - price)
# 2. delta from median price for that neighbourhood = (median_proce_neighbourhood - price)
# 3. north_south column 1/2 or north/south = dict_north_south {'Dublin 1': 'N', 'Dublin 2', 'S', ...}
# 4. days since ad published = difference between 2 ephocs (publishDate - today)
# 5. distance from city center = difference between lat, long (Haversine formula)
# *6. commute time to city centre by {walking/cycling/train/bus}
# 7. categorical column for price ranges {bins}
# 8. calculated field from 7 => num of properties in that neighbourhood for that price range
# 9. num_ {Pharmacies, Supermarkets, Restaurants, Cafes, Parks} in 5K radius
# 10. price per bedroom = (price / numBedrooms)

In [39]:
final_df2['pricePerBedroom'] = final_df2['price'] / final_df2['numBedrooms']

In [40]:
final_df2['avgPriceNeighbourhood'] = final_df2.groupby('neighbourhood')['price'].transform(np.average)

In [41]:
final_df2['medianPriceNeighbourhood'] = final_df2.groupby('neighbourhood')['price'].transform(np.median)

In [42]:
final_df2['deltaAvgPrice'] = final_df2['avgPriceNeighbourhood'] - final_df2['price']

In [43]:
final_df2['deltaMedianPrice'] = final_df2['medianPriceNeighbourhood'] - final_df2['price']

In [44]:
dict_north_south = {'Dublin 1':'N', 'Dublin 10':'S', 'Dublin 11':'N', 'Dublin 12':'S', 'Dublin 13':'N', 'Dublin 14':'S', 'Dublin 15':'N', 
                    'Dublin 16':'S', 'Dublin 17':'N', 'Dublin 18':'S', 'Dublin 2':'S', 'Dublin 20':'S', 'Dublin 22':'S', 'Dublin 24':'S', 
                    'Dublin 3':'N', 'Dublin 4':'S', 'Dublin 5':'N', 'Dublin 6':'S', 'Dublin 6W':'S', 'Dublin 7':'N', 'Dublin 8':'S', 'Dublin 9':'N'}

In [45]:
final_df2["neighbourhood"] = final_df2["neighbourhood"].str.strip()
final_df2['dublinNorthSouth']=final_df2['neighbourhood'].map(dict_north_south)

final_df2.groupby('dublinNorthSouth').size()

dublinNorthSouth
N    829
S    882
dtype: int64

In [46]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    All args must be of equal length.    
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [47]:
final_df2['distToCity'] = haversine_np(final_df2['longitude'], final_df2['latitude'], -6.2580, 53.3531)

In [48]:
final_df2.isna().sum()

title                       0
neighbourhood               0
propertyType                0
numBedrooms                 0
numBathrooms                0
floorArea                   0
price                       0
rating                      0
sellerId                    0
longitude                   0
latitude                    0
publishDate                 0
pricePerBedroom             0
avgPriceNeighbourhood       0
medianPriceNeighbourhood    0
deltaAvgPrice               0
deltaMedianPrice            0
dublinNorthSouth            0
distToCity                  0
dtype: int64

In [49]:
final_df2['date'] = pd.to_datetime(final_df2['publishDate'], unit='ms')
final_df2['daysSincePublished'] = pd.to_datetime("now") - final_df2['date']
final_df2['daysSincePublished'] = final_df2['daysSincePublished'].apply(lambda x: x.days)

In [50]:
zero_days = final_df2[final_df2['daysSincePublished'] == 0].index
final_df2.drop(zero_days , inplace=True)

final_df2.groupby(['daysSincePublished']).size()

daysSincePublished
1       5
2       6
3      14
4      16
5      14
6       1
8      17
9      14
10      2
11     17
12     14
14      3
15      5
16     19
17     12
18     13
19     16
20      3
21      5
22     19
23     12
24     23
25     15
26     14
28      2
29     11
30     13
31     16
32     10
33      9
34      1
36     12
37     10
38      5
39     13
40     14
41      1
42      3
43     19
44      6
45     16
46      8
47     10
50      9
51     14
52     10
53      5
54     11
56      3
57      6
58      8
59      9
60      7
61     11
63      3
64     11
65      7
66     24
67     22
68      8
69      2
71      7
72     14
73      8
74     10
75     10
76      2
77      1
78      3
79      3
80      7
81     12
82     31
83      1
84      2
86      3
88      3
91      9
93      6
94     10
95     19
96      6
98      2
99      9
100     8
101    17
102    23
103    15
105     5
106    11
107     7
108    13
109    13
110    16
111     1
112     1
113    11
114    21
1

In [51]:
final_df2.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,floorArea,price,rating,sellerId,longitude,latitude,publishDate,pricePerBedroom,avgPriceNeighbourhood,medianPriceNeighbourhood,deltaAvgPrice,deltaMedianPrice,dublinNorthSouth,distToCity,date,daysSincePublished
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Dublin 15,Apartment,2.0,2.0,65,200000.0,C1,1815,-6.392909,53.422384,1616077963000,100000.000000,344137.142857,285000.0,144137.142857,85000.0,N,11.799143,2021-03-18 14:32:43,9
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",Dublin 11,End of Terrace,3.0,1.0,95,260000.0,E2,1815,-6.319989,53.384121,1615453791000,86666.666667,291284.403670,275000.0,31284.403670,15000.0,N,5.364379,2021-03-11 09:09:51,16
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Dublin 15,Apartment,2.0,2.0,75,340000.0,B3,4274,-6.337658,53.373575,1616057065000,170000.000000,344137.142857,285000.0,4137.142857,-55000.0,N,5.751517,2021-03-18 08:44:25,9
12,"6 Sycamore Avenue, Castleknock, Dublin 15",Dublin 15,Semi-D,3.0,1.0,94,390000.0,D1,1590,-6.387025,53.375988,1616424638000,130000.000000,344137.142857,285000.0,-45862.857143,-105000.0,N,8.925828,2021-03-22 14:50:38,5
13,"5 Ellensborough Court, Kiltipper Road, Kiltipp...",Dublin 24,Semi-D,3.0,3.0,124,359000.0,C1,2608,-6.366592,53.268978,1615809829000,119666.666667,282918.604651,270000.0,-76081.395349,-89000.0,S,11.805436,2021-03-15 12:03:49,12


In [52]:
final_df3 = final_df2[['title','neighbourhood','propertyType','numBedrooms','numBathrooms','floorArea','price','rating','sellerId','longitude','latitude','pricePerBedroom','deltaAvgPrice','deltaMedianPrice','dublinNorthSouth','distToCity','daysSincePublished']]

In [53]:
final_df3.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,floorArea,price,rating,sellerId,longitude,latitude,pricePerBedroom,deltaAvgPrice,deltaMedianPrice,dublinNorthSouth,distToCity,daysSincePublished
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Dublin 15,Apartment,2.0,2.0,65,200000.0,C1,1815,-6.392909,53.422384,100000.000000,144137.142857,85000.0,N,11.799143,9
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",Dublin 11,End of Terrace,3.0,1.0,95,260000.0,E2,1815,-6.319989,53.384121,86666.666667,31284.403670,15000.0,N,5.364379,16
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Dublin 15,Apartment,2.0,2.0,75,340000.0,B3,4274,-6.337658,53.373575,170000.000000,4137.142857,-55000.0,N,5.751517,9
12,"6 Sycamore Avenue, Castleknock, Dublin 15",Dublin 15,Semi-D,3.0,1.0,94,390000.0,D1,1590,-6.387025,53.375988,130000.000000,-45862.857143,-105000.0,N,8.925828,5
13,"5 Ellensborough Court, Kiltipper Road, Kiltipp...",Dublin 24,Semi-D,3.0,3.0,124,359000.0,C1,2608,-6.366592,53.268978,119666.666667,-76081.395349,-89000.0,S,11.805436,12


In [54]:
# below is where we make use of the Foursquare API

In [55]:
CLIENT_ID = 'WV2XS4MH5YRWGHLTCFT4CKR4SRWNHWAF3JHWHNN4MKEQWTL3' # your Foursquare ID
CLIENT_SECRET = 'QEWWIOG0M3BT4V0YPNSKJY521MDUBHBYWBXCJFZ0452KP3OT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [56]:
# 1. Food 4d4b7105d754a06374d81259
# 2. Outdoors & Recreation 4d4b7105d754a06377d81259
# 3. Shop & Service 4d4b7105d754a06378d81259

In [57]:
food_cat_ids = ['4bf58dd8d48988d16d941735','4bf58dd8d48988d128941735','4bf58dd8d48988d1e0931735','4bf58dd8d48988d110941735',
                '4bf58dd8d48988d149941735','4bf58dd8d48988d1fa931735','4bf58dd8d48988d1c4941735','4bf58dd8d48988d145941735',
                '4bf58dd8d48988d11b941735','4bf58dd8d48988d16e941735','4bf58dd8d48988d1c5941735','4bf58dd8d48988d143941735',
                '4bf58dd8d48988d1ce941735','4bf58dd8d48988d10e951735','4bf58dd8d48988d1c9941735','4bf58dd8d48988d1ca941735',
                '4bf58dd8d48988d142941735','4bf58dd8d48988d11e941735','4bf58dd8d48988d16a941735','52e81612bcbc57f1066b79f1',
                '4bf58dd8d48988d155941735','4bf58dd8d48988d1f9941735','4bf58dd8d48988d10f941735','4bf58dd8d48988d1cc941735']

recreation_cat_ids = ['4bf58dd8d48988d163941735','4bf58dd8d48988d1e6941735','4bf58dd8d48988d176941735','4bf58dd8d48988d137941735',
                     '4bf58dd8d48988d164941735','4bf58dd8d48988d1e4931735','4bf58dd8d48988d1e0941735','4bf58dd8d48988d12d951735',
                     '4bf58dd8d48988d1e2941735','4bf58dd8d48988d165941735','56aa371be4b08b9a8d57353e','58daa1558bbb0b01f18ec1fd',
                     '4deefb944765f83613cdba6e','4e74f6cabd41c4836eac4c31','56aa371be4b08b9a8d573562','4bf58dd8d48988d15e941735']

shop_cat_ids = ['52f2ab2ebcbc57f1066b8b46','4bf58dd8d48988d103951735','4bf58dd8d48988d1f6941735','4bf58dd8d48988d1fd941735',
               '4d954b0ea243a5684a65b473','4bf58dd8d48988d114951735','4bf58dd8d48988d112951735','4bf58dd8d48988d118951735',
               '4bf58dd8d48988d1f2941735','5032833091d4c4b30a586d60','52dea92d3cf9994f4e043dbb','4bf58dd8d48988d10f951735',
               '4bf58dd8d48988d1f8941735','4bf58dd8d48988d122951735','4bf58dd8d48988d106951735','4bf58dd8d48988d108951735']

In [58]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
        num_food = 0
        num_recreation = 0
        num_shop = 0
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']

        for v in results:
            num_food += 1 if (v['venue']['categories'][0]['id'] in food_cat_ids) else 0
            num_recreation += 1 if (v['venue']['categories'][0]['id'] in recreation_cat_ids) else 0
            num_shop += 1 if (v['venue']['categories'][0]['id'] in shop_cat_ids) else 0
#         print(num_food, num_recreation, num_shop)
        
        # return only relevant information for each nearby venue
        venues_list.append([
            name, 
            num_food,
            num_recreation,
            num_shop])
#     print(venues_list)

    nearby_venues = pd.DataFrame([venue_list for venue_list in venues_list])
    nearby_venues.columns = ['title', 
                  'numFood',
                  'numRecreation',
                  'numShop']
    
    return(nearby_venues)

In [63]:
dublin_venues = getNearbyVenues(names=final_df3['title'],
                                   latitudes=final_df3['latitude'],
                                   longitudes=final_df3['longitude'])

In [64]:
dublin_venues.shape

(1182, 4)

In [65]:
dublin_venues.head()

,title,numFood,numRecreation,numShop
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",32,5,20
1,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",47,16,2
2,"Apartment 35, Cloonlara Square, Phoenix Park R...",40,10,13
3,"6 Sycamore Avenue, Castleknock, Dublin 15",44,9,23
4,"5 Ellensborough Court, Kiltipper Road, Kiltipp...",23,4,20


In [66]:
dublin_venues2 = final_df3.join(dublin_venues.set_index('title'), on='title')

dublin_venues2.shape

(1210, 20)

In [67]:
dublin_venues2 = dublin_venues2.drop_duplicates()

In [68]:
# use pandas-profiling for a quick EDA and to make sure the features are created as desired
profile = pd_prof.ProfileReport(dublin_venues2) 
profile.to_file("output.html")

Summarize dataset:   0%|          | 0/33 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/pandas_profiling/visualisation/plot.py:166: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)


Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [70]:
dublin_venues2.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,floorArea,price,rating,sellerId,longitude,latitude,pricePerBedroom,deltaAvgPrice,deltaMedianPrice,dublinNorthSouth,distToCity,daysSincePublished,numFood,numRecreation,numShop
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Dublin 15,Apartment,2.0,2.0,65,200000.0,C1,1815,-6.392909,53.422384,100000.000000,144137.142857,85000.0,N,11.799143,9,32,5,20
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",Dublin 11,End of Terrace,3.0,1.0,95,260000.0,E2,1815,-6.319989,53.384121,86666.666667,31284.403670,15000.0,N,5.364379,16,47,16,2
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Dublin 15,Apartment,2.0,2.0,75,340000.0,B3,4274,-6.337658,53.373575,170000.000000,4137.142857,-55000.0,N,5.751517,9,40,10,13
12,"6 Sycamore Avenue, Castleknock, Dublin 15",Dublin 15,Semi-D,3.0,1.0,94,390000.0,D1,1590,-6.387025,53.375988,130000.000000,-45862.857143,-105000.0,N,8.925828,5,44,9,23
13,"5 Ellensborough Court, Kiltipper Road, Kiltipp...",Dublin 24,Semi-D,3.0,3.0,124,359000.0,C1,2608,-6.366592,53.268978,119666.666667,-76081.395349,-89000.0,S,11.805436,12,23,4,20


In [71]:
print('There are {} unique property types.'.format(len(dublin_venues2['propertyType'].unique())))
print('There are {} unique ber ratings.'.format(len(dublin_venues2['rating'].unique())))

There are 9 unique property types.
There are 16 unique ber ratings.


In [72]:
dublin_venues2.shape

(1175, 20)

In [73]:
# one hot encoding for the 2 categorical features above
dublin_onehot2 = pd.get_dummies(dublin_venues2[['propertyType']], prefix="", prefix_sep="")

In [74]:
dublin_onehot2.head()

,Apartment,Bungalow,Detached,Duplex,End of Terrace,Semi-D,Site,Terrace,Townhouse
0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0
12,0,0,0,0,0,1,0,0,0
13,0,0,0,0,0,1,0,0,0


In [75]:
dublin_onehot3 = pd.get_dummies(dublin_venues2[['rating']], prefix="", prefix_sep="")

In [76]:
dublin_onehot3.head()

,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,E1,E2,F,G,SI_666,ZZZ
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
13,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [77]:
test_df4 = pd.concat([dublin_venues2, dublin_onehot2], axis=1)

In [78]:
test_df4 = pd.concat([test_df4, dublin_onehot3], axis=1)

In [79]:
test_df4.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,floorArea,price,rating,sellerId,longitude,latitude,pricePerBedroom,deltaAvgPrice,deltaMedianPrice,dublinNorthSouth,distToCity,daysSincePublished,numFood,numRecreation,numShop,Apartment,Bungalow,Detached,Duplex,End of Terrace,Semi-D,Site,Terrace,Townhouse,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,E1,E2,F,G,SI_666,ZZZ
0,"170 Mount Garrett Park, Tyrellstown, Tyrrelsto...",Dublin 15,Apartment,2.0,2.0,65,200000.0,C1,1815,-6.392909,53.422384,100000.000000,144137.142857,85000.0,N,11.799143,9,32,5,20,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,"Muir Cu, 6 Westwood Road, Finglas, Dublin 11",Dublin 11,End of Terrace,3.0,1.0,95,260000.0,E2,1815,-6.319989,53.384121,86666.666667,31284.403670,15000.0,N,5.364379,16,47,16,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,"Apartment 35, Cloonlara Square, Phoenix Park R...",Dublin 15,Apartment,2.0,2.0,75,340000.0,B3,4274,-6.337658,53.373575,170000.000000,4137.142857,-55000.0,N,5.751517,9,40,10,13,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
12,"6 Sycamore Avenue, Castleknock, Dublin 15",Dublin 15,Semi-D,3.0,1.0,94,390000.0,D1,1590,-6.387025,53.375988,130000.000000,-45862.857143,-105000.0,N,8.925828,5,44,9,23,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
13,"5 Ellensborough Court, Kiltipper Road, Kiltipp...",Dublin 24,Semi-D,3.0,3.0,124,359000.0,C1,2608,-6.366592,53.268978,119666.666667,-76081.395349,-89000.0,S,11.805436,12,23,4,20,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [86]:
# 1) Clustering using Property characteristics => [NumBedrooms, NumBathrooms, FloorArea]
test_df4["labels_1"] = DBSCAN(eps=0.01, min_samples=10).fit(test_df4[["numBedrooms", "numBathrooms", "floorArea"]].values).labels_
print(test_df4["labels_1"].unique())

[-1  0  1  2  3  4  5  6  7  8  9 10 11]


In [87]:
test_df4.groupby('labels_1').size()

labels_1
-1     1024
 0       10
 1       18
 2       15
 3       13
 4       13
 5       11
 6       13
 7       16
 8       10
 9       12
 10      10
 11      10
dtype: int64

In [88]:
noisy_labels = test_df4[test_df4['labels_1'] == -1].index
mapping_df2 = test_df4.drop(noisy_labels)

mapping_df2.groupby(['labels_1']).size()

labels_1
0     10
1     18
2     15
3     13
4     13
5     11
6     13
7     16
8     10
9     12
10    10
11    10
dtype: int64

In [89]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.2)

# set color scheme for the clusters
num_clusters = 12
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, title, cluster in zip(mapping_df2['latitude'], mapping_df2['longitude'], mapping_df2['title'], mapping_df2['labels_1']):
    label = folium.Popup(str(title) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=1.0).add_to(map_clusters)
       
map_clusters

In [98]:
## investigating an interesting cluster nicely spread across the map
test_df4[test_df4.labels_1 == 10]

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,floorArea,price,rating,sellerId,longitude,latitude,pricePerBedroom,deltaAvgPrice,deltaMedianPrice,dublinNorthSouth,distToCity,daysSincePublished,numFood,numRecreation,numShop,Apartment,Bungalow,Detached,Duplex,End of Terrace,Semi-D,Site,Terrace,Townhouse,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,E1,E2,F,G,SI_666,ZZZ,labels_1
679,"Apartment 63, Saint Peters Square, Phibsboroug...",Dublin 7,Apartment,2.0,1.0,63,265000.0,D2,607,-6.272357,53.361633,132500.0,112484.210526,85000.0,N,1.343799,38,57,9,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,10
793,"41 The Rectory, Stepaside, Dublin 18",Dublin 18,Apartment,2.0,1.0,63,260000.0,C2,2615,-6.219662,53.255898,130000.0,259395.061728,165000.0,S,11.097536,96,25,8,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,10
826,"Apartment 26, The Cedars, Herbert Park Lane, B...",Dublin 4,Apartment,2.0,1.0,63,425000.0,C3,329,-6.231967,53.327321,212500.0,273284.482759,70000.0,S,3.345127,60,53,14,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,10
985,"44 Deerpark Drive, Kiltipper, Dublin 24",Dublin 24,Apartment,2.0,1.0,63,185000.0,D1,10948,-6.376230,53.270464,92500.0,95890.109890,90000.0,S,12.080680,89,29,4,23,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,10
1164,"29 Green Isle Court, Clondalkin, Dublin 22",Dublin 22,Apartment,2.0,1.0,63,200000.0,D1,8891,-6.404264,53.310064,100000.0,42820.000000,27000.0,S,10.820576,93,40,4,34,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,10
1207,"23 The Park, Kingswood Heights, Kingswood, Dub...",Dublin 24,Bungalow,2.0,1.0,63,285000.0,G,463,-6.369618,53.304563,142500.0,-4109.890110,-10000.0,S,9.163264,90,41,5,30,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,10
1589,"Apartment 131, Inishtrahull Custom House Harbo...",Dublin 1,Apartment,2.0,1.0,63,350000.0,C3,10659,-6.246135,53.350963,175000.0,-33894.736842,-50000.0,N,0.822056,128,55,12,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,10
1592,"71 Addison Drive, Glasnevin, Dublin 11",Dublin 11,Apartment,2.0,1.0,63,295000.0,C2,9172,-6.276927,53.376131,147500.0,1078.260870,-16000.0,N,2.850493,71,58,11,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,10
2292,"9 Kearn's Court, Kearn's Place, Kilmainham, Du...",Dublin 8,Apartment,2.0,1.0,63,275000.0,C2,1413,-6.305082,53.341177,137500.0,65576.000000,25000.0,S,3.392727,90,48,11,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,10
2728,"3 Kilmainham Orchard, Turvey Avenue, Kilmainha...",Dublin 8,Apartment,2.0,1.0,63,250000.0,E1,2621,-6.311155,53.341144,125000.0,90576.000000,50000.0,S,3.768185,114,51,9,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,10


In [99]:
# 2) Clustering using Location characteristics => [Longitude, Latitude]
test_df4["labels_2"] = DBSCAN(eps=0.01, min_samples=10).fit(test_df4[["longitude", "latitude"]].values).labels_
print(test_df4["labels_2"].unique())

[ 0  1 -1  2  3  4  5  6  8  7  9]


In [100]:
test_df4.groupby('labels_2').size()

labels_2
-1    145
 0    815
 1     50
 2     35
 3     51
 4     56
 5     42
 6     12
 7     14
 8     10
 9      9
dtype: int64

In [101]:
noisy_labels = test_df4[test_df4['labels_2'] == -1].index
mapping_df2 = test_df4.drop(noisy_labels)

mapping_df2.groupby(['labels_2']).size()

labels_2
0    815
1     50
2     35
3     51
4     56
5     42
6     12
7     14
8     10
9      9
dtype: int64

In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.2)

# set color scheme for the clusters
num_clusters = 10
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, title, cluster in zip(mapping_df2['latitude'], mapping_df2['longitude'], mapping_df2['title'], mapping_df2['labels_2']):
    label = folium.Popup(str(title) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.3).add_to(map_clusters)
       
map_clusters

In [105]:
# 3) Clustering using Neighbourhood characteristics => [NumFood, NumRecreation, NumShop]
test_df4["labels_3"] = DBSCAN(eps=0.01, min_samples=5).fit(test_df4[["numFood", "numRecreation", "numShop"]].values).labels_
print(test_df4["labels_3"].unique())

[-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35]


In [106]:
test_df4.groupby('labels_3').size()

labels_3
-1     966
 0      11
 1       6
 2       5
 3      10
 4      12
 5       6
 6      10
 7       7
 8      15
 9       8
 10     10
 11      5
 12      5
 13      5
 14      9
 15      7
 16      6
 17      9
 18      6
 19      9
 20      5
 21     10
 22     11
 23      7
 24      5
 25      5
 26     11
 27     11
 28      5
 29      9
 30      5
 31      5
 32      6
 33      6
 34      6
 35      5
dtype: int64

In [107]:
noisy_labels = test_df4[test_df4['labels_3'] == -1].index
mapping_df2 = test_df4.drop(noisy_labels)

mapping_df2.groupby(['labels_3']).size()

labels_3
0     11
1      6
2      5
3     10
4     12
5      6
6     10
7      7
8     15
9      8
10    10
11     5
12     5
13     5
14     9
15     7
16     6
17     9
18     6
19     9
20     5
21    10
22    11
23     7
24     5
25     5
26    11
27    11
28     5
29     9
30     5
31     5
32     6
33     6
34     6
35     5
dtype: int64

In [110]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.2)

# set color scheme for the clusters
num_clusters = 38
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, title, cluster in zip(mapping_df2['latitude'], mapping_df2['longitude'], mapping_df2['title'], mapping_df2['labels_3']):
    label = folium.Popup(str(title) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

In [114]:
# investigating a cluster from Dublin 7, Smithfield/Stoneybatter
test_df4[test_df4.labels_3 == 4]

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,floorArea,price,rating,sellerId,longitude,latitude,pricePerBedroom,deltaAvgPrice,deltaMedianPrice,dublinNorthSouth,distToCity,daysSincePublished,numFood,numRecreation,numShop,Apartment,Bungalow,Detached,Duplex,End of Terrace,Semi-D,Site,Terrace,Townhouse,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,E1,E2,F,G,SI_666,ZZZ,labels_1,labels_2,labels_3
81,"15 Sarsfield Quay Apartments, Liffey Street We...",Dublin 7,Duplex,3.0,2.0,100,355000.0,C3,435,-6.284269,53.347117,118333.333333,22484.210526,-5000.0,N,1.865040,4,52,9,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,4
425,"41 Carnew Street, Stoneybatter, Dublin 7",Dublin 7,Terrace,2.0,2.0,80,450000.0,D1,6498,-6.292165,53.354527,225000.000000,-72515.789474,-100000.0,N,2.271619,41,52,9,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,4
539,"49 Clipper View, Eilis Quay, Smithfield, Dublin 7",Dublin 7,Apartment,2.0,1.0,53,245000.0,E2,11,-6.285053,53.347615,122500.000000,132484.210526,105000.0,N,1.895192,5,52,9,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,4
755,"126 Dolphin Road, Drimnagh, Dublin 12",Dublin 12,Terrace,2.0,1.0,68,285000.0,G,1413,-6.303333,53.334937,142500.000000,29821.428571,10000.0,S,3.622007,34,52,9,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5,0,4
1181,"Apartment 37, Harcourt Lodge, Inchicore, Dublin 8",Dublin 8,Apartment,2.0,1.0,60,275000.0,C2,10939,-6.312340,53.336446,137500.000000,65576.000000,25000.0,S,4.052285,116,52,9,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,7,0,4
1393,"10 Church Avenue South, Rialto, Dublin 8",Dublin 8,Terrace,2.0,1.0,75,425000.0,D2,2013,-6.294583,53.335852,212500.000000,-84424.000000,-125000.0,S,3.092565,11,52,9,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,4
1607,"Apartment 45, Collins Square, Benburb Street, ...",Dublin 7,Apartment,1.0,1.0,48,260000.0,D2,6223,-6.284349,53.348074,260000.000000,117484.210526,90000.0,N,1.834863,6,52,9,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,4
1656,"7 Temple Court, Palatine Square, Stoneybatter,...",Dublin 7,Terrace,2.0,1.0,67,385000.0,C3,1087,-6.290173,53.351171,192500.000000,-7515.789474,-35000.0,N,2.144776,93,52,9,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,4
1928,"6 Saint Bricin's Park, Arbour Hill, Dublin 7",Dublin 7,Terrace,3.0,2.0,72,395000.0,E2,1087,-6.290769,53.349727,131666.666667,-17515.789474,-45000.0,N,2.205688,53,52,9,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,4
2184,"4 Church Avenue South, Rialto, Dublin 8",Dublin 8,Terrace,2.0,1.0,76,450000.0,D1,2013,-6.294676,53.335728,225000.000000,-109424.000000,-150000.0,S,3.105958,11,52,9,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,4


In [90]:
# the final section below creates some price buckets (deciles) as an additional price related feature and prepares the dataframe to be exported as a csv which will be used for some further visual analysis using Tableau

In [115]:
test_df4.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,floorArea,price,rating,sellerId,longitude,latitude,pricePerBedroom,deltaAvgPrice,deltaMedianPrice,dublinNorthSouth,distToCity,daysSincePublished,numFood,numRecreation,numShop,Apartment,Bungalow,Detached,Duplex,End of Terrace,Semi-D,Site,Terrace,Townhouse,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,E1,E2,F,G,SI_666,ZZZ,labels_1,labels_2,labels_3
3,"1 Moatfield Park, Coolock, Artane, Dublin 5",Dublin 5,Semi-D,3.0,1.0,91,380000.0,E2,453,-6.193117,53.388871,126666.666667,17221.153846,0.0,N,5.857172,13,49,17,11,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1
12,"150 Broadford Rise, Ballinteer, Dublin 16",Dublin 16,Semi-D,3.0,2.0,102,495000.0,C3,49,-6.261108,53.277830,165000.000000,10582.089552,-20000.0,S,8.366932,11,47,9,14,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1
14,"Apartment 172, Block C, Dublin 7",Dublin 7,Apartment,1.0,1.0,51,250000.0,C1,1331,-6.277584,53.348715,250000.000000,127484.210526,100000.0,N,1.387431,17,53,9,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0
15,"1 Fitzgibbon Lane, Dublin 1",Dublin 1,Terrace,1.0,1.0,60,140000.0,G,11902,-6.256604,53.357853,140000.000000,176105.263158,160000.0,N,0.536232,3,56,11,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,1
16,"18A Fitzgibbon Street, Dublin 1",Dublin 1,Terrace,1.0,1.0,53,240000.0,D2,11902,-6.256600,53.357783,240000.000000,76105.263158,60000.0,N,0.528618,3,56,11,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1


In [116]:
test_df4['price_decile'] = pd.qcut(test_df4['price'], 10, labels=False)

In [117]:
test_df4.groupby('price_decile').size()

price_decile
0    127
1    125
2    145
3    100
4    127
5    121
6    142
7    104
8    124
9    124
dtype: int64

In [118]:
test_df4.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,floorArea,price,rating,sellerId,longitude,latitude,pricePerBedroom,deltaAvgPrice,deltaMedianPrice,dublinNorthSouth,distToCity,daysSincePublished,numFood,numRecreation,numShop,Apartment,Bungalow,Detached,Duplex,End of Terrace,Semi-D,Site,Terrace,Townhouse,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,E1,E2,F,G,SI_666,ZZZ,labels_1,labels_2,labels_3,price_decile
3,"1 Moatfield Park, Coolock, Artane, Dublin 5",Dublin 5,Semi-D,3.0,1.0,91,380000.0,E2,453,-6.193117,53.388871,126666.666667,17221.153846,0.0,N,5.857172,13,49,17,11,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,6
12,"150 Broadford Rise, Ballinteer, Dublin 16",Dublin 16,Semi-D,3.0,2.0,102,495000.0,C3,49,-6.261108,53.277830,165000.000000,10582.089552,-20000.0,S,8.366932,11,47,9,14,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,8
14,"Apartment 172, Block C, Dublin 7",Dublin 7,Apartment,1.0,1.0,51,250000.0,C1,1331,-6.277584,53.348715,250000.000000,127484.210526,100000.0,N,1.387431,17,53,9,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,2
15,"1 Fitzgibbon Lane, Dublin 1",Dublin 1,Terrace,1.0,1.0,60,140000.0,G,11902,-6.256604,53.357853,140000.000000,176105.263158,160000.0,N,0.536232,3,56,11,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0
16,"18A Fitzgibbon Street, Dublin 1",Dublin 1,Terrace,1.0,1.0,53,240000.0,D2,11902,-6.256600,53.357783,240000.000000,76105.263158,60000.0,N,0.528618,3,56,11,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,1


In [119]:
def get_price_bucket (row):
    if row['price_decile'] < 3:
        return 'Low'
    elif row['price_decile'] < 7:
        return 'Medium'
    return 'High'

In [120]:
test_df4['price_bucket'] = test_df4.apply(lambda row: get_price_bucket(row), axis=1)

In [121]:
test_df4.groupby('price_bucket').size()

price_bucket
High      352
Low       397
Medium    490
dtype: int64

In [122]:
test_df4.groupby('price_bucket')['price'].mean()

price_bucket
High      640122.159091
Low       229370.277078
Medium    348577.551020
Name: price, dtype: float64

In [123]:
test_df4.head()

,title,neighbourhood,propertyType,numBedrooms,numBathrooms,floorArea,price,rating,sellerId,longitude,latitude,pricePerBedroom,deltaAvgPrice,deltaMedianPrice,dublinNorthSouth,distToCity,daysSincePublished,numFood,numRecreation,numShop,Apartment,Bungalow,Detached,Duplex,End of Terrace,Semi-D,Site,Terrace,Townhouse,A2,A3,B1,B2,B3,C1,C2,C3,D1,D2,E1,E2,F,G,SI_666,ZZZ,labels_1,labels_2,labels_3,price_decile,price_bucket
3,"1 Moatfield Park, Coolock, Artane, Dublin 5",Dublin 5,Semi-D,3.0,1.0,91,380000.0,E2,453,-6.193117,53.388871,126666.666667,17221.153846,0.0,N,5.857172,13,49,17,11,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1,0,-1,6,Medium
12,"150 Broadford Rise, Ballinteer, Dublin 16",Dublin 16,Semi-D,3.0,2.0,102,495000.0,C3,49,-6.261108,53.277830,165000.000000,10582.089552,-20000.0,S,8.366932,11,47,9,14,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,-1,8,High
14,"Apartment 172, Block C, Dublin 7",Dublin 7,Apartment,1.0,1.0,51,250000.0,C1,1331,-6.277584,53.348715,250000.000000,127484.210526,100000.0,N,1.387431,17,53,9,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,2,Low
15,"1 Fitzgibbon Lane, Dublin 1",Dublin 1,Terrace,1.0,1.0,60,140000.0,G,11902,-6.256604,53.357853,140000.000000,176105.263158,160000.0,N,0.536232,3,56,11,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1,0,1,0,Low
16,"18A Fitzgibbon Street, Dublin 1",Dublin 1,Terrace,1.0,1.0,53,240000.0,D2,11902,-6.256600,53.357783,240000.000000,76105.263158,60000.0,N,0.528618,3,56,11,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,1,1,Low


In [221]:
test_df4.to_csv('output.csv', index=False)